# 🤖 Telegram Bot для обработки видео

Этот блокнот запускает Telegram бота с поддержкой GPU для обработки видео.

**Важно:** Убедитесь, что включен GPU в настройках Colab:
- Меню → Среда выполнения → Сменить среду выполнения → Аппаратный ускоритель → GPU

## 📁 Шаг 1: Клонирование репозитория

In [ ]:
import os
import shutil

repo_path = '/content/telegram-bot'
repo_url = 'https://github.com/LeoSkln/telegram-bot-video-processing.git'

# Удаляем папку если существует
if os.path.exists(repo_path):
    print(f'Удаляем существующую директорию: {repo_path}')
    shutil.rmtree(repo_path)

# Клонируем репозиторий
print(f'Клонируем репозиторий из {repo_url}...')
!git clone {repo_url} {repo_path}

# Переходим в папку проекта
os.chdir(repo_path)

print(f'✅ Репозиторий клонирован в: {os.getcwd()}')
print('
📋 Файлы в проекте:')
!ls -la

## 📦 Шаг 2: Установка зависимостей

In [ ]:
# Установка системных зависимостей (ffmpeg)
print('Обновляем пакеты и устанавливаем ffmpeg...')
!apt-get update -qq
!apt-get install -y ffmpeg
print('✅ ffmpeg установлен!')

In [ ]:
# Устанавливаем PyTorch с CUDA поддержкой для GPU ускорения
print('\n🎮 Устанавливаем PyTorch с CUDA для GPU ускорения...')
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
print('✅ PyTorch для GPU установлен!')

In [ ]:
# Устанавливаем остальные зависимости из requirements.txt
print('
🐍 Устанавливаем зависимости из requirements.txt...')
!pip install --upgrade pip -q
!pip install -r requirements.txt -q

# Обновляем yt-dlp до самой последней версии
print('\n📦 Обновляем yt-dlp до последней версии...')
!pip install --upgrade --force-reinstall yt-dlp -q
print('✅ Все зависимости установлены и обновлены!')

In [ ]:
# Проверяем GPU и ffmpeg
import nest_asyncio
nest_asyncio.apply()

print('='*50)
print('🔍 ПРОВЕРКА ОКРУЖЕНИЯ')
print('='*50)

print('\n🎮 Проверяем GPU поддержку:')
try:
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f'✅ GPU доступен: {gpu_name}')
    else:
        print('❌ GPU недоступен, будет использоваться CPU')
except ImportError:
    print('❌ PyTorch не установлен')

print('
ffmpeg версия:')
!ffmpeg -version

print('
✅ Проверка окружения завершена!')

## ⚙️ Шаг 3: Настройка секретов

Для безопасной работы бота, пожалуйста, добавьте ваши токены как секреты в Google Colab.

1.  Нажмите на иконку **ключа (🔑)** на левой панели.
2.  Нажмите **+ Добавить новый секрет**.
3.  Создайте два секрета:
    - **`TELEGRAM_BOT_TOKEN`**: вставьте сюда токен вашего Telegram бота.
    - **`GOOGLE_OAUTH_TOKEN_BASE64`**: вставьте сюда ваш Google OAuth токен в формате Base64.
4.  Убедитесь, что переключатель **'Доступно в этом блокноте'** включен для обоих секретов.

In [ ]:
# Настройка переменных окружения из секретов Colab
import os
from google.colab import userdata

try:
    TELEGRAM_TOKEN = userdata.get('TELEGRAM_BOT_TOKEN')
    GOOGLE_TOKEN = userdata.get('GOOGLE_OAUTH_TOKEN_BASE64')
    
    if not TELEGRAM_TOKEN or not GOOGLE_TOKEN:
        raise ValueError('Не удалось получить секреты. Убедитесь, что вы их добавили.')
        
    os.environ['TELEGRAM_BOT_TOKEN'] = TELEGRAM_TOKEN
    os.environ['GOOGLE_OAUTH_TOKEN_BASE64'] = GOOGLE_TOKEN
    
    # Создаем .env файл для совместимости с кодом бота
    env_content = f'''TELEGRAM_BOT_TOKEN={TELEGRAM_TOKEN}
GOOGLE_OAUTH_TOKEN_BASE64={GOOGLE_TOKEN}
'''
    
    with open('.env', 'w') as f:
        f.write(env_content)
    
    print('✅ Переменные окружения настроены!')
    print('📄 Создан файл .env')

except Exception as e:
    print(f'❌ ОШИБКА: {e}')
    print('Пожалуйста, убедитесь, что вы правильно настроили секреты на предыдущем шаге.')

## 🍪 Шаг 4: Загрузка cookies (опционально)

Для безопасной работы бота, пожалуйста, добавьте ваши токены как секреты в Google Colab.

1.  Нажмите на иконку **ключа (🔑)** на левой панели.
2.  Нажмите **+ Добавить новый секрет**.
3.  Создайте два секрета:
    - **`TELEGRAM_BOT_TOKEN`**: вставьте сюда токен вашего Telegram бота.
    - **`GOOGLE_OAUTH_TOKEN_BASE64`**: вставьте сюда ваш Google OAuth токен в формате Base64.
4.  Убедитесь, что переключатель **'Доступно в этом блокноте'** включен для обоих секретов.

In [ ]:
# Загрузка cookies файла для yt-dlp
from google.colab import files
import os

print('🍪 Загрузите ваш cookies.txt файл для YouTube:')
print('ℹ️ Этот файл нужен для скачивания приватных или возрастных видео.')
print('ℹ️ Если у вас нет cookies файла, просто отмените загрузку.')

try:
    uploaded = files.upload()
    
    cookies_file = None
    for filename in uploaded.keys():
        if 'cookies' in filename.lower() or filename.endswith('.txt'):
            cookies_file = filename
            break
    
    if cookies_file:
        # Переименовываем в стандартное имя
        if cookies_file != 'cookies.txt':
            os.rename(cookies_file, 'cookies.txt')
        
        print(f'✅ Cookies файл загружен: cookies.txt')
        
        # Добавляем путь к cookies в .env файл
        cookies_path = os.path.join(os.getcwd(), 'cookies.txt')
        with open('.env', 'a') as f:
            f.write(f'
COOKIES_FILE={cookies_path}
')
        
        print('📄 Путь к cookies добавлен в .env')
    else:
        print('⚠️ Cookies файл не найден среди загруженных. Бот будет работать без них.')
        
except Exception as e:
    if 'aborted' in str(e):
        print('ℹ️ Загрузка отменена. Бот будет работать без cookies.')
    else:
        print(f'⚠️ Ошибка при загрузке cookies: {e}')

## 🚀 Шаг 5: Запуск бота

In [ ]:
# Запуск бота
import os

print('🤖 Запускаем Telegram бота...')
print(f'📍 Текущая директория: {os.getcwd()}')

try:
    # Запускаем основной файл бота
    if os.path.exists('run_bot.py'):
        print('🔄 Запускаем через run_bot.py')
        !python run_bot.py
    elif os.path.exists('bot.py'):
        print('🔄 Запускаем bot.py напрямую')
        !python bot.py
    else:
        print('❌ Не найден файл бота для запуска (run_bot.py или bot.py)')
        
except KeyboardInterrupt:
    print('
⏹️ Бот остановлен пользователем.')

## 📊 Просмотр логов (опционально)

Вы можете запустить эту ячейку, чтобы посмотреть последние логи работы бота.

In [ ]:
# Просмотр логов бота
import os

if os.path.exists('bot.log'):
    print('📋 Последние 50 строк из bot.log:')
    !tail -50 bot.log
else:
    print('📋 Файл логов bot.log не найден.') 